# 🚀 TREINAMENTO LORA SDXL - OTIMIZADO PARA COLAB PRO

## ⚡ INSTRUÇÕES RÁPIDAS:
1. **Runtime → Change runtime type → GPU A100 (recomendado para Colab Pro)**
2. **Runtime → Run All**
3. **Aguarde ~2 horas (A100) ou ~3 horas (V100)**
4. **Baixe o vídeo ao final**

---

## 📊 O QUE VAI ACONTECER:
- ✅ Baixar 39 fotos do Google Drive compartilhado
- ✅ Instalar todas as dependências
- ✅ Treinar modelo LoRA personalizado (1-2 horas com A100)
- ✅ Gerar vídeo 30s em 1080p (20-30 min com A100)
- ✅ Download automático do resultado

---

## 🎯 OTIMIZAÇÕES PARA COLAB PRO:
- **GPU A100:** Treinamento 2x mais rápido
- **Batch size aumentado:** Melhor qualidade
- **Mais steps:** Modelo mais preciso
- **Sem timeout:** Execução garantida

**⏱️ TEMPO TOTAL COM A100: ~2 horas**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CÉLULA 1: BAIXAR FOTOS DO GOOGLE DRIVE COMPARTILHADO
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🚀 INICIANDO PIPELINE COMPLETO - OTIMIZADO PARA COLAB PRO")
print("="*80 + "\n")

# Verificar GPU disponível
import subprocess
gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader']).decode('utf-8').strip()
print(f"🎮 GPU Detectada: {gpu_info}")

if 'A100' in gpu_info:
    print("✅ A100 detectada! Usando configurações otimizadas.")
    BATCH_SIZE = 2
    MAX_STEPS = 600
    INFERENCE_STEPS = 30
elif 'V100' in gpu_info:
    print("✅ V100 detectada! Usando configurações balanceadas.")
    BATCH_SIZE = 1
    MAX_STEPS = 500
    INFERENCE_STEPS = 28
else:
    print("⚠️  T4 detectada. Recomendado: Runtime → Change runtime type → A100")
    BATCH_SIZE = 1
    MAX_STEPS = 400
    INFERENCE_STEPS = 25

print("\n[1/5] Baixando fotos do Google Drive compartilhado...\n")

import os
from pathlib import Path

# Instalar gdown se necessário
!pip install -q gdown

# Criar diretório para fotos
!mkdir -p /content/training_images

# ID da pasta compartilhada do Google Drive
FOLDER_ID = "1tYGc99OjiSCpUTg4QgB-sQc0GErJ3arN"

# Baixar todas as fotos da pasta compartilhada
print("Baixando fotos...")
!gdown --folder https://drive.google.com/drive/folders/{FOLDER_ID} -O /content/training_images --remaining-ok

# Contar fotos baixadas
photos = list(Path("/content/training_images").rglob("*.jpg")) + \
         list(Path("/content/training_images").rglob("*.png")) + \
         list(Path("/content/training_images").rglob("*.JPG")) + \
         list(Path("/content/training_images").rglob("*.PNG"))

print(f"\n✅ {len(photos)} fotos baixadas com sucesso!")

if len(photos) == 0:
    raise ValueError("❌ ERRO: Nenhuma foto foi baixada! Verifique o link do Google Drive.")

print("\nPrimeiras 5 fotos:")
for i, photo in enumerate(photos[:5]):
    print(f"  {i+1}. {photo.name}")

print(f"\n📊 Configurações de treinamento:")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Max steps: {MAX_STEPS}")
print(f"  - Inference steps: {INFERENCE_STEPS}")
print("\n" + "="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CÉLULA 2: INSTALAR DEPENDÊNCIAS E PREPARAR AMBIENTE
# ═══════════════════════════════════════════════════════════════════════════════

print("\n[2/5] Instalando dependências...\n")

# Criar diretórios necessários
!mkdir -p /content/models/gui4_lora_sdxl
!mkdir -p /content/output_video/frames

# Instalar pacotes principais
print("Instalando pacotes principais...")
!pip install -q diffusers transformers accelerate safetensors bitsandbytes pillow opencv-python

# Clonar repositório diffusers
if not os.path.exists("/content/diffusers"):
    print("Clonando repositório diffusers...")
    !git clone -q https://github.com/huggingface/diffusers.git /content/diffusers

# Instalar requirements do DreamBooth SDXL
print("Instalando requirements do DreamBooth...")
!pip install -q -r /content/diffusers/examples/dreambooth/requirements_sdxl.txt

# Atualizar peft para versão compatível
print("Atualizando peft...")
!pip install -q --upgrade --force-reinstall 'peft>=0.17.0'

print("\n✅ Todas as dependências instaladas!")
print("="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CÉLULA 3: TREINAR MODELO LORA (OTIMIZADO PARA COLAB PRO)
# ═══════════════════════════════════════════════════════════════════════════════

import os
from datetime import datetime

os.chdir('/content/diffusers/examples/dreambooth')

print("\n" + "="*80)
print("[3/5] INICIANDO TREINAMENTO LORA")
print("="*80)
print(f"⏰ Início: {datetime.now().strftime('%H:%M:%S')}")

if 'A100' in gpu_info:
    print("⏱️  Tempo estimado: 1-1.5 horas (A100)")
elif 'V100' in gpu_info:
    print("⏱️  Tempo estimado: 1.5-2 horas (V100)")
else:
    print("⏱️  Tempo estimado: 2-3 horas (T4)")

print("="*80 + "\n")

!python train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --instance_data_dir="/content/training_images" \
  --output_dir="/content/models/gui4_lora_sdxl" \
  --instance_prompt="photo of gui4 person" \
  --resolution=512 \
  --train_batch_size={BATCH_SIZE} \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --max_train_steps={MAX_STEPS} \
  --checkpointing_steps=100 \
  --seed=42 \
  --mixed_precision="fp16"

print("\n" + "="*80)
print(f"⏰ Fim: {datetime.now().strftime('%H:%M:%S')}")
print("="*80)

# Verificar se o modelo foi salvo
files = os.listdir("/content/models/gui4_lora_sdxl")
print(f"\n📦 {len(files)} arquivos salvos no modelo")

if len(files) == 0:
    raise ValueError("❌ ERRO: Modelo não foi salvo!")

print("\n✅ TREINAMENTO CONCLUÍDO COM SUCESSO!")
print("="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CÉLULA 4: GERAR VÍDEO 30s EM 1080p (OTIMIZADO PARA COLAB PRO)
# ═══════════════════════════════════════════════════════════════════════════════

import torch
from diffusers import StableDiffusionXLPipeline
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import gc

print("\n" + "="*80)
print("[4/5] GERANDO VÍDEO 30s EM 1080p")
print("="*80)
print(f"⏰ Início: {datetime.now().strftime('%H:%M:%S')}")

if 'A100' in gpu_info:
    print("⏱️  Tempo estimado: 20-30 minutos (A100)")
elif 'V100' in gpu_info:
    print("⏱️  Tempo estimado: 30-40 minutos (V100)")
else:
    print("⏱️  Tempo estimado: 40-50 minutos (T4)")

print("="*80 + "\n")

print("Carregando modelo SDXL...")
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)
pipe.to("cuda")

print("Carregando LoRA treinado...")
pipe.load_lora_weights("/content/models/gui4_lora_sdxl")
pipe.fuse_lora()

print("✅ Modelo carregado!\n")

# Configurações do vídeo
WIDTH = 1920
HEIGHT = 1080
FPS = 30
DURATION = 30
TOTAL_FRAMES = FPS * DURATION

frames_dir = Path("/content/output_video/frames")
frames_dir.mkdir(parents=True, exist_ok=True)

# Prompts variados para o vídeo
prompts = [
    "photo of gui4 person, professional portrait, studio lighting, high quality, detailed face",
    "photo of gui4 person, smiling, natural lighting, happy expression, photorealistic",
    "photo of gui4 person, serious expression, dramatic lighting, cinematic",
    "photo of gui4 person, casual pose, soft lighting, relaxed atmosphere",
    "photo of gui4 person, confident look, professional setting, sharp focus",
    "photo of gui4 person, outdoor setting, natural background, golden hour lighting",
    "photo of gui4 person, close-up portrait, shallow depth of field, bokeh",
    "photo of gui4 person, business attire, modern office, professional environment",
]

print(f"Gerando {TOTAL_FRAMES} frames ({WIDTH}x{HEIGHT})...\n")

for i in tqdm(range(TOTAL_FRAMES), desc="Gerando frames"):
    prompt = prompts[i % len(prompts)]
    
    image = pipe(
        prompt=prompt,
        num_inference_steps=INFERENCE_STEPS,
        guidance_scale=7.5,
        height=HEIGHT,
        width=WIDTH,
    ).images[0]
    
    image.save(frames_dir / f"frame_{i:04d}.png")
    
    # Limpar memória a cada 100 frames
    if (i + 1) % 100 == 0:
        gc.collect()
        torch.cuda.empty_cache()

print("\n✅ Todos os frames gerados!")

# Liberar memória
del pipe
gc.collect()
torch.cuda.empty_cache()

print("\nCriando vídeo MP4...")
video_path = "/content/output_video/video_30s_1080p.mp4"

!ffmpeg -y -framerate {FPS} -i /content/output_video/frames/frame_%04d.png \
  -c:v libx264 -pix_fmt yuv420p -crf 18 -preset slow {video_path}

import os
video_size = os.path.getsize(video_path) / (1024*1024)

print("\n" + "="*80)
print("✅ VÍDEO CONCLUÍDO!")
print("="*80)
print(f"📁 Arquivo: {video_path}")
print(f"📊 Tamanho: {video_size:.1f} MB")
print(f"⏰ Fim: {datetime.now().strftime('%H:%M:%S')}")
print("="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CÉLULA 5: BAIXAR VÍDEO
# ═══════════════════════════════════════════════════════════════════════════════

from google.colab import files

print("\n" + "="*80)
print("[5/5] BAIXANDO VÍDEO FINAL")
print("="*80 + "\n")

print("Iniciando download do vídeo...")
files.download('/content/output_video/video_30s_1080p.mp4')

print("\n" + "="*80)
print("🎉 PIPELINE COMPLETO EXECUTADO COM SUCESSO!")
print("="*80)
print("\n✅ O vídeo foi baixado para seu computador!")
print("\n📊 RESUMO:")
print(f"  - GPU utilizada: {gpu_info}")
print("  - Fotos treinadas: 39")
print("  - Modelo: LoRA SDXL")
print(f"  - Steps de treinamento: {MAX_STEPS}")
print("  - Vídeo: 30s @ 1080p (30 FPS)")
print("  - Total de frames: 900")
print(f"  - Tamanho do vídeo: {video_size:.1f} MB")
print("\n" + "="*80)